In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

warnings.filterwarnings("ignore", category = RuntimeWarning)

### Exploratory Data Analysis (EDA)

In [ ]:
filepath = r'.\data\housing.csv'
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df = pd.read_csv(filepath, sep = '\s+', header = None, names = column_names)
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Box and Whisker Plots
fig, axes = plt.subplots(nrows = 2, ncols = 7, figsize = (15, 6))

axes = axes.flatten()

for i, column in enumerate(df.columns):
    sns.boxplot(y = df[column], data = df, ax = axes[i], color = '#14F278')

plt.tight_layout()
plt.show()

In [ ]:
# Scatter Plots
fig, axes = plt.subplots(nrows = 2, ncols = 7, figsize = (15, 6))

axes = axes.flatten()

for i, column in enumerate(df.columns):
    sns.regplot(x = df[column], y = df['MEDV'], data = df, ax = axes[i], color = '#14F278', line_kws = dict(color = '#064824'))

plt.tight_layout()
plt.show()

In [ ]:
# Pairplot
pairplot = sns.pairplot(df, kind = 'reg', plot_kws={'color': '#14F278', 'line_kws': {'color': '#064824'}}, diag_kws={'color': '#14F278'})
plt.show()

In [ ]:
# We wanted to explore MEDV further as it is our target value. We noticed there for RM vs MEDV that there are a few towns with the same MEDV but very different RM values.
# After further investigation we learned that every MEDV value above 50 was rounded down to 50, so we decided to remove those rows.
df = df[~(df['MEDV'] >= 50.0)].reset_index(drop = True)
df.shape

In [ ]:
# Correlation Heatmap
corr_df = df.corr().abs()

fig, ax = plt.subplots(figsize = (15, 10))
sns.heatmap(corr_df, annot = True, cmap = 'BuGn', linewidths = 0.5)
plt.show()

In [ ]:
# Covariance Heatmap
cov_df = df.cov()

fig, ax = plt.subplots(figsize = (15, 10))
sns.heatmap(cov_df, annot = True, cmap = 'BuGn', linewidths = 0.5, center = 0, robust = True)
plt.show()

In [ ]:
# Lasso Regression
X = df.drop(columns = ['MEDV'])
y = df['MEDV']
 
x_train, x_test, y_train, y_test = train_test_split(X, y,  test_size = 0.20, random_state = 42)
 
model = Lasso(alpha = 1)
model.fit(x_train, y_train)
y_pred1 = model.predict(x_test)
 
lasso = pd.DataFrame()
lasso['Columns'] = x_train.columns
lasso['Estimate'] = pd.Series(model.coef_)
 
sns.barplot(x = lasso['Columns'], y = lasso['Estimate'], color = '#14F278')
plt.xticks(rotation = 45)
plt.show()

### Our Model - Using sklearn

Gradient Boosted Trees (GBT) is a popular machine learning technique used for both regression and classification tasks. It's an ensamble learning method, meaning it combines the predictions of another model to improve overall performance.

Gradient boosting is a sequential technique where new models are added to correct the errors made by existing models. In each iteration, a new decision tree model is trained to predict the residuals (the differences between the actual values and the predictions of the ensemble so far).

A decision tree is a flowchart-like structure where each internal node represents a feature, each branch represents a decision based on that feature, and each leaf node represents the outcome or prediction.

The "gradient" in Gradient Boosting refers to the gradient descent optimization algorithm. It's used to minimize the loss function of the ensemble model. Gradient descent adjusts the parameters of the new model in the direction that reduces the loss the most.

Each new decision tree model is added to the ensemble, with its predictions weighted by a factor (learning rate) that determines how much influence it has on the final prediction. The learning rate is typically a small value, and it helps to prevent overfitting and stabilize the training process.

The process of adding new models continues until a predefined number of trees are added or until a stopping criterion is met, such as the maximum depth of the tree or the minimum number of samples in each node.

Overall, Gradient Boosted Trees are powerful and widely used due to their ability to capture complex relationships in data, handle missing values, and provide high predictive accuracy.


Based on our EDA, we've decided to use RM, PTRATIO, TAX, and LSTAT as the features to predict the MEDV values.

- RM:
- PTRATIO:
- TAX:
- LSTAT:

In [ ]:
X = df[['RM', 'PTRATIO', 'TAX', 'LSTAT']]
y = df['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Initialize an instance of a Gradient Boosting Regression
model = GradientBoostingRegressor()

Hyperparameter tuning is the process of selecting the best set of hyperparameters for a machine learning algorithm to optimize its performance on a give dataset.

To find the best set of hyperparameters for our model, we decided to use GridSearchCV. GridSearchCV (Grid Search Cross-Validation) searches through a manually specified subset of hyperparameter combinations to find the best set for a given model.

In [ ]:
# Pass Ranges for Hyperparameters
parameters = {'max_depth': [2, 3, 4],
              'min_samples_split': [4, 5, 6],
              'learning_rate': [0.05, 0.1, 0.2],
              'n_estimators': [25, 50, 75]}  

# Initialise a Grid Search object
clf = GridSearchCV(estimator = model, param_grid = parameters, cv = 5)

# Fit the Grid Search object to the train data
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

In [ ]:
# Retrieve the optimal hyperparamer values, found by Grid Search CV
best_depth = clf.best_estimator_.max_depth
best_min_samples = clf.best_estimator_.min_samples_leaf

print(f'best estimator: depth {best_depth} and min_samples {best_min_samples}')

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean absolute error is {mae:.4f}')
print(f'MSE (test): {mse:.4f}')
print(f'RMSE (test): {mse**0.5:.4f}')
print(f'R-squared is {r2:.4f}')

### Our Model - From Scratch

In [ ]:
class Node():
    """This class defines the attributes of a tree node."""
    def __init__(self, feature = None, threshold = None, left = None, right = None, value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

In [ ]:
class DecisionTree():
    """Define a decision tree regressor.
    
    Args:
        min_samples (int, optional): The minimum number of samples required to split an
        internal node. Defaults to 5.
        max_depth (int, optional): The maximum depth of the tree. Defaults to 3.
    """
    def __init__(self, min_samples: int = 5, max_depth: int = 3):
        self.min_samples = min_samples
        self.max_depth = max_depth
        self.root = None

    def fit(self, X: pd.DataFrame, y: pd.DataFrame):
        """Build a decision tree regressor from the training set (X, y)."""
        X = X.to_numpy()
        self.root = self._grow_tree(X, y)

    def predict(self, X: pd.DataFrame) -> list:
        """Predict the y values."""
        X = X.to_numpy()
        return [self._transverse(x, self.root) for x in X]

    def _transverse(self, x: np.array, node: Node) -> float:
        """Retrieve the value of y."""
        if node.value is None:
            if x[node.feature] <= node.threshold:
                return self._transverse(x, node.left)
            return self._transverse(x, node.right)
        return node.value

    def _grow_tree(self, X: np.array, y: np.array, depth: int = 0) -> Node:
        """Split the data into leaf nodes."""
        n_samples = X.shape[0]
        if n_samples >= self.min_samples and depth <= self.max_depth:
            index, value = self._best_split(X, y)
            left_mask = X[:, index] <= value
            right_mask = X[:, index] > value

            left = self._grow_tree(X[left_mask], y[left_mask], depth + 1)
            right = self._grow_tree(X[right_mask], y[right_mask], depth + 1)

            return Node(feature = index, threshold = value, left = left, right = right)

        return Node(value = self._leaf_node(y))

    def _best_split(self, X: pd.DataFrame, y: pd.DataFrame) -> tuple:
        """Find the best threshold and index to split the node on."""
        best_rss = float('inf')
        best_feature = None
        best_threshold = None
        n_features = X.shape[1]

        for feature_i in range(n_features):
            for threshold in np.unique(X[:, feature_i]):
                left_mask = X[:, feature_i] <= threshold
                right_mask = X[:, feature_i] > threshold
                rss = self._rss(y, left_mask, right_mask)
                if rss < best_rss:
                    best_rss = rss
                    best_feature = feature_i
                    best_threshold = threshold

        return best_feature, best_threshold

    def _rss(self, y: pd.DataFrame, left, right) -> float:
        """Calculate the Residual Sum of Squares (RSS)."""
        y_left = y[left]
        y_right = y[right]
        rss_left = np.sum((y_left - np.mean(y_left)) ** 2)
        rss_right = np.sum((y_right - np.mean(y_right)) ** 2)
        return rss_left + rss_right

    def _leaf_node(self, y: pd.DataFrame) -> float:
        """Calculate the value of a leaf node."""
        return np.mean(y)

In [ ]:
class ScratchGradientBoostingRegressor():
    """Define a gradient boosting regressor."""
    def __init__(self, max_depth = 3, min_samples = 5, learning_rate = 0.1, n_estimators = 50):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators

        self.trees = []
        for _ in range(n_estimators):
            tree = DecisionTree(min_samples = self.min_samples, max_depth = self.max_depth)
            self.trees.append(tree)

    def fit(self, X: pd.DataFrame, y):
        """Build a gradient boosting regressor from the training set (X, y)."""
        y_pred = np.copy(y)
        for tree in self.trees:
            tree.fit(X, y_pred)
            new_y_pred = tree.predict(X)
            y_pred = y_pred - np.multiply(self.learning_rate, new_y_pred)

    def predict(self, X):
        """Predict the y values."""
        y_pred = np.zeros(X.shape[0])
        for tree in self.trees:
            new_y_pred = tree.predict(X)
            y_pred += np.multiply(self.learning_rate, new_y_pred)
        return y_pred

In [ ]:
def mae_val(y, y_pred):
    """Calculate Mean Absolute Error."""
    return np.mean(abs(y - y_pred))

def mse_val(y, y_pred):
    """Calculate Mean Square Error."""
    return np.mean(np.square(np.subtract(y, y_pred)))

def r_square(y, y_pred):
    """Calculate R-Squared."""
    ssr = np.sum(np.square(np.subtract(y, y_pred)))
    sst = np.sum(np.square(np.subtract(y, np.mean(y))))
    return 1 - ssr/sst

def performance_evaluation(y, y_pred):
    """Print all the performance metrics."""
    mae = mae_val(y, y_pred)
    mse = mse_val(y, y_pred)
    r2 = r_square(y, y_pred)

    print(f'Mean absolute error is {mae:.4f}')
    print(f'MSE (test): {mse:.4f}')
    print(f'RMSE (test): {mse**0.5:.4f}')
    print(f'R-squared is {r2:.4f}')

In [ ]:
def train_test(X, y, train_size: float = 0.8, seed: int = None):
    """Split the dataset into train and tests sets."""
    if seed is not None:
        np.random.seed(seed)

    num_samples = len(X)
    index = np.arange(num_samples)
    np.random.shuffle(index)

    X = X.loc[index]
    y = y.loc[index]

    split_i = int(num_samples * train_size)
    X_train, X_test = X[:split_i], X[split_i:]
    y_train, y_test = y[:split_i], y[split_i:]

    return X_train, X_test, y_train, y_test

In [ ]:
import itertools

def grid_search(parameters, X_train, X_test, y_train, y_test):
    best_r2 = 0

    param_combinations = list(itertools.product(*parameters.values()))

    for combo in param_combinations:
        model = ScratchGradientBoostingRegressor(**dict(zip(parameters.keys(), combo)))
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2 = r_square(y_test, y_pred)
        if r2 > best_r2:
            best_r2 = r2
            best_combo = combo
            best_y_pred = y_pred

    return best_r2, best_combo, best_y_pred

In [ ]:
X_train, X_test, y_train, y_test = train_test(X, y, train_size = 0.8, seed = 42)

parameters = {'max_depth': [2, 3, 4],
              'min_samples': [4, 5, 6],
              'learning_rate': [0.05, 0.1, 0.2],
              'n_estimators': [25, 50, 75]} 

best_r2, best_params, y_pred = grid_search(parameters, X_train, X_test, y_train, y_test)

In [ ]:
performance_evaluation(y_test, y_pred)

sns.scatterplot(x = y_test, y = y_pred, color = '#14F278')

plt.xlabel("Actual Values (y_test)")
plt.ylabel("Predicted Values (y_pred)")
plt.title("Scatter Plot of Actual vs. Predicted Values")

plt.show()